Импортируем библиотеки

In [1]:
import numpy as np
import time
import random
!pip install pycuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.curandom import rand as curandom
from pycuda import driver as dr
from pycuda.compiler import SourceModule

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 4.6 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=d7d30fc08acdbd243c77fa6fdeeae3b1d674a6b2c3f32580c1c618e062799edf
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1-py2.py3-none-any.whl size=63426 sha256=32d6ff465afaa0ee93cf585bb9fdb5a3131fb81011ac7c42074e657dcd4b0c77
  Stored in directory: /root/.cache/pip/wheels/b4/11/0e/75dfe0884733df6a47170da81d3f1b28276ff7bc4315287190
Successfully built pycuda pytools


Зададим количество элементов после запятой

In [18]:
N = 10000

Зададим функцию для выполнения на ядре

In [19]:
kernel = SourceModule(
    """
    __global__ void foundpi(double *x, double *y, int *points_in_circle, const int N){
        int j = blockIdx.x * blockDim.x + threadIdx.x;
        int c = 0;
        for (int i = j; i < N; i += gridDim.x * blockDim.x) {
            if (x[i]*x[i] + y[i]*y[i] <= 1) {
            c+=1;
            }
        }
        atomicAdd(points_in_circle, c);
    }
    """)

Зададим функцию, которая будет проводить рассчёты числа Pi методом Монте-Карло на CPU и GPU, а также выводить все необходимые метрики

<!-- Используем N - количество чисел после запятой, чтобы определить количество раз, сколько необходимо исполнить цикл для вычисления числа pi. В этой же функции пропишем реализацию вычислений на CPU и GPU соответственно -->

In [32]:
def main():    
  n_vals = []
  cpu_ts = []
  gpu_ts = []
  np_ts = []
  acc = []
  acc_np_gpu_vals = []
  acc_cpu_np_vals = []
  data = [cpu_ts, gpu_ts, np_ts, n_vals, acc, acc_np_gpu_vals, acc_cpu_np_vals]
  for power in range(14, 22):  # степени 2
    N = 2 ** power  # power, it's a power! OnePuMa
    print(N, f"2^{power}")
    start_time = time.time()
    points_in_circle = 0
    x = np.zeros((N, 1), dtype=np.float64)
    y = np.zeros((N, 1), dtype=np.float64)
    for i in range(N):
      x[i] = random.uniform(-1, 1)
      y[i] = random.uniform(-1, 1)

    for i in range(N):
      if x[i] ** 2 + y[i] ** 2 <= 1:
        points_in_circle = points_in_circle + 1

    pi = 4 * points_in_circle / N
    end_time = time.time()
    cpu_t = end_time - start_time
    err = np.abs(np.pi - pi)

    
    start_time = time.time()
    points_in_circle = 0
    np_x = np.random.uniform(-1,1,(N, 1))
    np_y = np.random.uniform(-1,1,(N, 1))
    
    points_in_circle_np = np.count_nonzero(np_x ** 2 + np_y ** 2 <=1)

    np_pi = 4 * points_in_circle_np / N
    end_time = time.time()
    np_t = end_time - start_time
    err_np = np.abs(np.pi - np_pi)


    start_time = time.time()
    gpu_points_in_circle = gpuarray.zeros((1,), dtype=np.int32)
    gpu_points_in_circle = gpu_points_in_circle.get()

    gpu_x = curandom((N,), dtype=np.double).get().astype(np.double)
    gpu_y = curandom((N,), dtype=np.double).get().astype(np.double)
    pi_calc = kernel.get_function("foundpi")
    pi_calc(dr.In(gpu_x), dr.In(gpu_y), dr.Out(gpu_points_in_circle), np.int32(N), block=(128, 1, 1),
            grid=(int(N / (128 ** 2)), 1))
    dr.Context.synchronize()

    gpu_pi = 4 * gpu_points_in_circle[0] / N
    end_time = time.time()
    gpu_t = end_time - start_time
    gpu_err = np.abs(np.pi - gpu_pi)

    acceleration = cpu_t / gpu_t

    print(pi)
    print('Cpu time: ', cpu_t, '. Error: ', err)

    print(pi)
    print('np time: ', np_t, '. Error: ', abs(np_pi - np.pi))
    acc_np_gpu = np_t / gpu_t
    acc_cpu_np = cpu_t / np_t
    print('T(np)/T(gpu):', acc_np_gpu)
    print('T(cpu)/T(np):', acc_cpu_np)

    print(gpu_pi)
    print('Gpu time: ', gpu_t, '. Error: ', gpu_err)
    print('T(cpu)/T(gpu):', acceleration,"\n")

    cpu_ts.append(cpu_t)
    gpu_ts.append(gpu_t)
    n_vals.append(N)
    acc.append(acceleration)
    np_ts.append(np_t)
    acc_np_gpu_vals.append(acc_np_gpu)
    acc_cpu_np_vals.append(acc_cpu_np)

  print(data)

Запустим заданную функцию

In [33]:
if __name__ == '__main__':
  main()

16384
3.1474609375
Cpu time:  0.12485051155090332 . Error:  0.005868283910206884
3.1474609375
np time:  0.0008020401000976562 . Error:  0.001944216089793116
T(np)/T(gpu): 0.0038113106648720424
T(cpu)/T(np): 155.66617122473247
3.134521484375
Gpu time:  0.21043682098388672 . Error:  0.007071169214793116
T(cpu)/T(gpu): 0.5932921385486203 

32768
3.1480712890625
Cpu time:  0.21938276290893555 . Error:  0.006478635472706884
3.1480712890625
np time:  0.0015473365783691406 . Error:  0.003164919214793116
T(np)/T(gpu): 0.00750519813029209
T(cpu)/T(np): 141.78089368258858
3.140380859375
Gpu time:  0.20616865158081055 . Error:  0.001211794214793116
T(cpu)/T(gpu): 1.0640936981777056 

65536
3.146484375
Cpu time:  0.4593162536621094 . Error:  0.004891721410206884
3.146484375
np time:  0.0032334327697753906 . Error:  0.002572385472706884
T(np)/T(gpu): 0.01571025116477345
T(cpu)/T(np): 142.0522046895738
3.13385009765625
Gpu time:  0.20581674575805664 . Error:  0.007742555933543116
T(cpu)/T(gpu): 2.23